In [1]:
!pip3 install pprint
from pprint import pprint

In [2]:
!pip3 install beautifulsoup4

In [3]:
!pip3 install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.7/site-packages (20.1)


In [4]:
!pip3 install requests

In [5]:
from six.moves.urllib.parse import quote as _quote
from bs4 import BeautifulSoup as _BeautifulSoup
import requests as _requests

__BASE_URL__ = 'https://lyrics.wikia.com'

In [6]:
class LyricsNotFound(Exception):
    __module__ = Exception.__module__

    def __init__(self, message=None):
        super(LyricsNotFound, self).__init__(message)

In [7]:
def urlize(string):
    """Convert string to LyricWikia format"""
    return _quote('_'.join(string.title().split()))

In [8]:
def create_url(artist, song, language):
    """Create the URL in the LyricWikia format"""
    url = __BASE_URL__ + '/wiki/{artist}:{song}'.format(artist=urlize(artist), song=urlize(song))
    if language:
        url += '/{language}'.format(language=urlize(language).lower())
    return url

In [21]:
def get_lyrics_by_language(artist, song, language, linesep='\n', timeout=None):
    """Retrieve the lyrics of the song in a particular language and return the first one in case
    multiple versions are available."""
    return get_all_lyrics(artist, song, language, linesep, timeout)[0]

In [17]:
def get_lyrics(artist, song, language='', linesep='\n', timeout=None):
    """Retrieve the lyrics of the song and return the first one in case
    multiple versions are available."""
    return get_all_lyrics(artist, song, language, linesep, timeout)[0]

In [18]:

def get_all_lyrics(artist, song, language='', linesep=' \n ', timeout=None):
    """Retrieve a list of all the lyrics versions of a song."""
    url = create_url(artist, song, language)
    response = _requests.get(url, timeout=timeout)
    soup = _BeautifulSoup(response.content, "html.parser")
    lyricboxes = soup.findAll('div', {'class': 'lyricbox'})

    if not lyricboxes:
        raise LyricsNotFound('Cannot download lyrics')

    for lyricbox in lyricboxes:
        for br in lyricbox.findAll('br'):
            br.replace_with(linesep)

    return [lyricbox.text.strip() for lyricbox in lyricboxes]

In [11]:
class Song(object):
    """A Song backed by the LyricWikia API"""

    def __init__(self, artist, title):
        self.artist = artist
        self.title = title

    @property
    def lyrics(self):
        """Song lyrics obtained by parsing the LyricWikia page"""
        return get_lyrics(self.artist, self.title,'')

    def __str__(self):
        return "Song(artist='%s', title='%s')" % (self.artist, self.title)

    def __repr__(self):
        return str(self)

In [12]:
class Album(object):
    """An Album backed by the LyricWikia API"""

    def __init__(self, artist, album_data):
        self.artist = artist
        self.title = album_data['album']
        self.year = album_data['year']
        self.songs = [Song(artist, song) for song in album_data['songs']]

    def __str__(self):
        return "Album(artist='%s', title='%s')" % (self.artist, self.title)

    def __repr__(self):
        return str(self)

In [13]:

class Artist(object):
    """An Artist backed by the LyricWikia API"""

    __API__ = __BASE_URL__ + '/api.php?fmt=json&func=getArtist&artist={artist}'

    def __init__(self, name):
        url = self.__API__.format(artist=urlize(name))
        data = _requests.get(url).json()
        self.name = data['artist']
        self.albums = [Album(self.name, album) for album in data['albums']]

    def __str__(self):
        return "Artist(name='%s')" % (self.name)

    def __repr__(self):
        return str(self)

In [26]:
lyrics = get_lyrics('one piece','Binks\' Sake')

In [24]:
pprint (lyrics)

('Yohohoho, yohohoho\n'
 'Yohohoho, yohohoho\n'
 'Yohohoho, yohohoho\n'
 'Yohohoho, yohohoho\n'
 '\n'
 'Binkusu no sake wo\n'
 'Todoke ni yuku yo\n'
 'Umikaze kimakase namimakase\n'
 'Shio no mukou de\n'
 'Yuuhi mo sawagu\n'
 'Sora nya wa wo kaku\n'
 'Tori no uta\n'
 'Sayonara minato\n'
 'Tsumugi no sato yo\n'
 'DON to icchou utao\n'
 'Funade no uta\n'
 'Kinpa-ginpa mo shibuki ni kaete\n'
 'Oretachya yuku zo\n'
 'Umi no kagiri\n'
 '\n'
 'Binkusu no sake wo\n'
 'Todoke ni yuku yo\n'
 'Warera kaizoku\n'
 'Umi watteku\n'
 'Nami wo makura ni\n'
 'Negura wa fune yo\n'
 'Ho ni hata ni ketateru wa dokuro\n'
 'Arashi ga kita zo\n'
 'Senri no sora ni\n'
 'Nami ga odoru yo\n'
 'DORAMU narase\n'
 'Okubyoukaze ni fukakerya saigo\n'
 'Asu no asahi ga nai ja nashi\n'
 '\n'
 'Yohohoho, yohohoho\n'
 'Yohohoho, yohohoho\n'
 'Yohohoho, yohohoho\n'
 'Yohohoho, yohohoho\n'
 '\n'
 'Binkusu no sake wo\n'
 'Todoke ni yuku yo\n'
 'Kyou ka asu ka to yoi no yume\n'
 'Te wo furu kage ni\n'
 'Mou aenai yo\n'
 'Na